In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import cv2
import numpy as np
from tqdm import tqdm

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Globals

In [2]:
metadata_dict = dict()
img_list = list()

# label_list is a list of np_arrays
label_list = list()

#### Read in label metadata

In [3]:
with open("fileinfo.csv", "r") as metafile:
    header = True
    
    for line in metafile:
        if header is True:
            header = False
            continue
        
        line_data = line.strip("\n").split(",")
        
        # Create a dictionary containing info of current image
        current_image_data = dict()
        filename = line_data[0]
        current_image_data['topLeftX'] = line_data[1]
        current_image_data['topLeftY'] = line_data[2]
        current_image_data['bottomRightX'] = line_data[3]
        current_image_data['bottomRightY'] = line_data[4]
        
        # Add current_image_data to metadata_dict
        metadata_dict[filename] = current_image_data

#### Create training & test set

In [4]:
img_path = '/home/david/Documents/CHECK_FUN/check_dataset_CURRENT/img/'

In [5]:
for filename in os.listdir(img_path):
    if filename.endswith(".jpg"):
        full_image_filepath = os.path.join(img_path, filename)
        
        # Note that cv2 reads in images as numpy arrays by default
        image = cv2.imread(full_image_filepath)
        
        topLeftX = float((metadata_dict[filename]['topLeftX']))/300.0
        topLeftY = float((metadata_dict[filename]['topLeftY']))/150.0
        bottomRightX = float((metadata_dict[filename]['bottomRightX']))/300.0
        bottomRightY = float((metadata_dict[filename]['bottomRightY']))/150.0
        
        
        # Add the current image to image_list...
        img_list.append(image)
        
        # ... followed by its respective label to label_list.
        img_metadata = np.asarray((topLeftX,
                                   topLeftY,
                                   bottomRightX,
                                   bottomRightY))
        label_list.append(img_metadata)

#### Normalize metadata
We want all of the widths and heights to be normalized between 0 and 1.

In [6]:
split_ratio = int(len(img_list) * 0.8)

In [7]:
# input data
x_train = np.array(img_list[:split_ratio])
x_test = np.array(img_list[split_ratio:])

# label data
y_train = np.array(label_list[:split_ratio])
y_test = np.array(label_list[split_ratio:])

#### Create Model

In [8]:
batch_size = 32
epochs = 100
image_width = 300
image_height = 150
my_kernel_size = (3, 3)
num_filters = 32
num_outputs = 4

In [9]:
model = Sequential()
model.add(Conv2D(num_filters, my_kernel_size, padding='same', input_shape=(image_height,image_width, 3)))
model.add(Activation('relu'))
model.add(Conv2D(num_filters, my_kernel_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering="tf"))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_outputs))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(data_format="channels_last", pool_size=(2, 2))`
  


In [10]:
model.fit(
    x_train, 
    y_train, 
    epochs=40, 
    verbose=1)

Epoch 1/40
97/97 [==============================] - 19s 199ms/step - loss: 166620344.6881 - mean_absolute_error: 7992.1360
Epoch 2/40
97/97 [==============================] - 17s 178ms/step - loss: 3298354.0941 - mean_absolute_error: 1471.7808
Epoch 3/40


KeyboardInterrupt: 